# Importing packages

In [14]:

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from tensorflow.python.client import device_lib
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import tensorflow_hub as hub
import shutil
import random
from PIL import Image
import pickle
import shutil

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
print(tf.__version__)

# IMG_SHAPE = 224
IMG_SHAPE = 512

2.8.0


# Resizing the images

In [16]:
# load every image in the food folder and resize them to 224x224, then save them in a new folder

for filename in os.listdir('food'):
    img = Image.open('food/' + filename)
    img = img.resize((IMG_SHAPE, IMG_SHAPE))
    img.save('food_'+str(IMG_SHAPE)+'/' + filename)


# everything below is no longer needed (it was used to extract features with mobilenet, to get that 0.61 score)

# Extracting the features

In [17]:
# URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
# URL = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"
URL = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/feature_vector/2"

feature_extractor = hub.KerasLayer(URL, input_shape=(IMG_SHAPE, IMG_SHAPE,3))
feature_extractor.trainable = False

model = tf.keras.Sequential([
  feature_extractor,
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 1280)              207615832 
                                                                 
Total params: 207,615,832
Trainable params: 0
Non-trainable params: 207,615,832
_________________________________________________________________


In [18]:
img = np.array(Image.open('food_512/00001.jpg'))[None]
print(img.shape)
output = model.predict(img)
print(output.shape)

(1, 512, 512, 3)
(1, 1280)


In [19]:

for filename in os.listdir('food_512'):
    name = filename.replace(".jpg","")
    if(name != "02461"):
        continue

    img = Image.open('food_512/' + filename)
    img = np.array(img)*1.0/255
    feature = model.predict(img[None])[0]
    print(feature)


[ 0.14465111  0.09083702  0.41782254 ...  2.7458491   1.6808972
 -0.00417358]


In [21]:
# dictionary with all the image names and their features
features = {}

# convert every image into features
line_id = 0
for filename in os.listdir('food_512'):
    line_id += 1
    img = Image.open('food_512/' + filename)
    img = np.array(img)*1.0/255
    feature = model.predict(img[None])[0]
    features[filename.replace(".jpg","")] = feature
    if line_id % 100 == 0:
        print(line_id)
    # break

# save the features dictionary to a file using pickle
with open('features_xl.pickle', 'wb') as handle:
    pickle.dump(features, handle, protocol=pickle.HIGHEST_PROTOCOL)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000


In [22]:
output_shape = 1280
features = {}
# load the features dictionary from the file
with open('features_xl.pickle', 'rb') as handle:
    features = pickle.load(handle)
print(features['02461'])

[ 0.14465111  0.09083702  0.41782254 ...  2.7458491   1.6808972
 -0.00417358]


In [24]:
print(len(features))
print(len(features["00000"]))

10000
1280


In [8]:
# each line of train_triplets.txt contains 3 images, separated by a space
# the first 2 images are the anchor and positive examples, and the third is the negative example
# extract the features of the 3 images and save them in a file
# cols = columns=np.array(['x' + str(i) for i in np.arange(output_shape*2)+1])
# df_features = pd.DataFrame(columns=cols)
# df_labels = pd.DataFrame(columns=['label'])

# line_id = 0
# with open('train_triplets.txt', 'r') as f:
#     # go through the lines
#     for line in f:
#         line_id += 1
#         line = line.replace("\n", "")
#         anchor = features[line.split(' ')[0]]
#         positive = features[line.split(' ')[1]]
#         negative = features[line.split(' ')[2]]
#         if random.random() < 0.5:
#             row = np.concatenate((positive-anchor, negative-anchor))
#             df_labels = df_labels.append(pd.DataFrame(np.array([[1]]), columns=['label']))
#         else:
#             row = np.concatenate((negative-anchor, positive-anchor))
#             df_labels = df_labels.append(pd.DataFrame(np.array([[0]]), columns=['label']))
#         # add the new row to the dataframe
#         df_features = df_features.append(pd.DataFrame(row.reshape(1, -1), columns=cols))
#         if line_id % 5000 == 0:
#             print(line_id)
#             # break
    
# print(df_features.shape)
# print(df_labels.shape)
# df_features.to_csv('train_features.csv',index=False, float_format='%.3f')
# df_labels.to_csv('train_labels.csv',index=False)
